In [4]:
#!pip install gensim
#!pip install python-Levenshtein

In [5]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords 
from nltk import word_tokenize #tokenizar
import emoji 
from sklearn.feature_extraction.text import CountVectorizer #TF
from sklearn.feature_extraction.text import TfidfVectorizer #Tf-idf
from string import punctuation #signos de puntuacion
from sklearn.model_selection import train_test_split #otorgar pesos para w2v
from gensim.models import word2vec #para aplicar word2vec

In [7]:
df = pd.read_excel("DATOS_UNICOS_TOTALES.xlsx")
df.head()

,HATE,COMENTARIO
0,0.0,¿Ya vieron que en la #AO habrá evento por el #...
1,0.0,muy XD que para el primer punto exiges que Yam...
2,0.0,Apoco lees?\n1-El primer punto es para hablar ...
3,0.0,vos se nota que no tenes compresión lectora xD...
4,0.0,"No, literalmente Kaido y todos los piratas bes..."


In [8]:
file = open('stopwords1.txt', 'r', encoding = 'utf-8')
stopwords_1 = file.read()

In [9]:
non_words = list(punctuation) #lista de signos de puntuacion
#we add spanish punctuation
non_words.extend(['¿', '¡']) #se extiende la lista para agregar dos signos mas
non_words.extend(map(str,range(10))) #se extiende la lista para agregar numeros del 1 al 10

In [10]:
#limpieza de texto
def clean(text):
    text = emoji.demojize(str(text)) #convierte los emojis en texto
    urls = []
    for i in text.split():
        if not i.startswith('http'): #se quitan las palabras que empiecen con http
            urls.append(i)
    text1 = ' '.join(urls)
    cl1 = re.findall(r'[^¿@#&?\w+][\w+]+',text1) #quitamos caracteres especiales
    text2 = ' '.join(cl1)
    cl3 = re.findall(r'[a-zA-Záéíóúñ\s:_]+',text2) #nos quedamos con palabras que contengan el siguiente patrón
    text3 = ' '.join(cl3)
    text4 = text.lower()
    c = text4.maketrans('áéíóú','aeiou') #se cambia las vocales con tílde a vocales sin tilde
    text5 = text4.translate(c)
    text = re.sub(r"lgbt", "", text5) #se reemplaza la etiqueta lgbt
    text = re.sub(r"lgbti", "", text) # se reemplaza la etiqueta lgbti
    #L = [palabra for palabra in text5.split() if len(palabra) >2 and palabra not in stopwords_1]
    return ''.join(text)

In [11]:
def tokenize(text): #tokenizar cada comentario
    text = clean(text) #se aplica la limpieza anterior
    text = re.sub(r"http\S+", "", text) #se vuelve a limpiar para reemplazar las palabras que tengan ese patrón
    text = re.sub(r":", "", text) #se reemplaza : por nada
    text = ''.join([c for c in text if c not in non_words]) #Se quitan todos los signos de puntuacion incluidas en la librería
    text = re.sub(r'(.)\1+', r'\1', text) #se remueven caracteres repetidos
    tokens =  word_tokenize(text) #se tokeniza
    return ' '.join(tokens) #para w2v se junta las palabras sino se usa solo return tokens tf-idf


In [12]:
df['Texto Limpio'] = df['COMENTARIO'].apply(tokenize) #se aplica la función tokenize que incluye la limpieza a cada comentario
df

,HATE,COMENTARIO,Texto Limpio
0,0.0,¿Ya vieron que en la #AO habrá evento por el #...,ya vieron que en la ao habra evento por el pri...
1,0.0,muy XD que para el primer punto exiges que Yam...,muy xd que para el primer punto exiges que yam...
2,0.0,Apoco lees?\n1-El primer punto es para hablar ...,apoco les el primer punto es para hablar de po...
3,0.0,vos se nota que no tenes compresión lectora xD...,vos se nota que no tenes compresion lectora xd...
4,0.0,"No, literalmente Kaido y todos los piratas bes...",no literalmente kaido y todos los piratas best...
...,...,...,...
14274,0.0,felizmente o jiang cheng não é gay\n\né pan e ...,felizmente o jiang cheng não e gay e pan e ace...
14275,0.0,VOTACIÓN SUPER MASIVA \n\nCOMENTEN 100 o 20...,votacion super masiva comenten o veces meta co...
14276,0.0,A portarnos mal mal mal,a portarnos mal mal mal
14277,0.0,El sábado 26 ya podemos ir sin mascarilla...\n...,el sabado ya podemos ir sin mascarila alguien ...


## WORD2VEC

In [20]:
df.head()

,HATE,COMENTARIO,Texto Limpio
0,0.0,¿Ya vieron que en la #AO habrá evento por el #...,ya vieron que en la ao habra evento por el pri...
1,0.0,muy XD que para el primer punto exiges que Yam...,muy xd que para el primer punto exiges que yam...
2,0.0,Apoco lees?\n1-El primer punto es para hablar ...,apoco les el primer punto es para hablar de po...
3,0.0,vos se nota que no tenes compresión lectora xD...,vos se nota que no tenes compresion lectora xd...
4,0.0,"No, literalmente Kaido y todos los piratas bes...",no literalmente kaido y todos los piratas best...


In [21]:
#clean(df['Texto Limpio'].values[6949])

In [22]:
df2 = pd.DataFrame(df["Texto Limpio"].values)
np.savetxt("Comentarios_limpios_W2V.txt",df2,fmt="%s",encoding="utf-8") #se guarda el dataframe limpio para que luego sea usado en word2vec

In [23]:
#Se separa en dos dataframes, uno con discursos etiquetados como odio y otro con los comentarios de no odio.
df_no_hate = df[df["HATE"]==0]
df_hate = df[df["HATE"]==1]

In [24]:
df_hate

,HATE,COMENTARIO,Texto Limpio
15,1.0,Esto es la máxima imbecilidad,esto es la maxima imbecilidad
46,1.0,No nos engañemos. Los homosexuales bien sabemo...,no nos engañemos los homosexuales bien sabemos...
52,1.0,"Claro que es ilegal.\nSi esa fuera la razón, c...",claro que es ilegal si esa fuera la razon clar...
89,1.0,A los curas pro MOVIMIENTO LGTBI+ deberían ser...,a los curas pro movimiento lgtbi deberian ser ...
90,1.0,"A los de siempre. Hablo del Movimiento, de la ...",a los de siempre hablo del movimiento de la ap...
...,...,...,...
14157,1.0,@13Azulcrema Y eso que cabrón? Si allá está pe...,azulcrema y eso que cabron si ala esta penado ...
14181,1.0,"Cuchame, soy la mierda más delulu que te puede...",cuchame soy la mierda mas delulu que te puedes...
14182,1.0,@NEXWAL999 tampoco te las harías pq eres gay,nexwal tampoco te las harias pq eres gay
14203,1.0,Not mi amiga diciéndome que ha ido a comer sol...,not mi amiga diciendome que ha ido a comer sol...


In [25]:
df_hate.shape, df_no_hate.shape #Se puede observar las dimensiones de cada dataframe

((504, 3), (13766, 3))

### Separando train / teste (0.8 - 0.2)

In [133]:
#Se saca los comentarios clasificados como hate para el entrenamiento (80%)
hate_train = df_hate[:403].sample(400)
hate_train.dropna(how="any",inplace=True)
hate_train.shape

(400, 3)

In [134]:
#Se saca los comentarios clasificados como hate para el testeo (20%)
hate_test = df_hate[403:].sample(100)
hate_test.dropna(how="any",inplace=True)
hate_test.shape

(100, 3)

In [135]:
#Se saca los comentarios clasificados como no hate para el entrenamiento (80%)
no_hate_train = df_no_hate[:10000].sample(400)
no_hate_train.dropna(how="any",inplace=True)
no_hate_train.shape

(400, 3)

In [136]:
#Se saca los comentarios clasificados como no hate para el testeo (20%)
no_hate_test = df_no_hate[10000:].sample(100)
no_hate_test.dropna(how="any",inplace=True)
no_hate_test.shape

(100, 3)

In [137]:
no_hate_train

,HATE,COMENTARIO,Texto Limpio
1030,0.0,es que recuerdo por ejemplo personajes de la p...,es que recuerdo por ejemplo personajes de la p...
9971,0.0,Hace rato andaba de chismosa en un chat gay y ...,hace rato andaba de chismosa en un chat gay y ...
2138,0.0,asique discriminando a los osos eh...,asique discriminando a los osos eh
7048,0.0,"@MiguelA45290236 @MYMALK4PON3 No soy gay, es p...",miguela mymalkpon no soy gay es para apoyar a ...
4554,0.0,Menuda mierder,menuda mierder
...,...,...,...
2863,0.0,Igualmente estoy de acuerdo con las personas q...,igualmente estoy de acuerdo con las personas q...
1636,0.0,CHAU FLFNFLFNFMKD\n@Y4T0SKHLOROM,chau flfnflfnfmkd ytskhlorom
5130,0.0,Esto es una pesadilla. Juegan con nuestra volu...,esto es una pesadila juegan con nuestra volunt...
2826,0.0,"Literal a nadie le importa una mierda,ni que h...",literal a nadie le importa una mierdani que hu...


### Aplicando W2V

In [138]:
lineas = word2vec.Text8Corpus("Comentarios_limpios_W2V.txt") #Se usa el txt anteriormente guardado que presenta los datos limpios
modelo_w2v = word2vec.Word2Vec(lineas,vector_size=100,min_count=2) #Se aplica la tecnica word2vec

In [139]:
modelo_w2v.wv['odio']

array([-0.21714494,  0.7032939 ,  0.09981456,  0.16321196,  0.2636095 ,
       -0.7439507 ,  0.34549475,  1.3506867 , -0.6539837 , -0.6171259 ,
       -0.14615037, -0.6687141 ,  0.10245548,  0.3436508 ,  0.24853788,
       -0.5598931 ,  0.28244442, -0.1687612 , -0.4046003 , -1.2275983 ,
        0.4587972 ,  0.14262539,  0.8236008 , -0.41751045, -0.14681716,
        0.07000034, -0.45931533, -0.08844616, -0.57858753, -0.04277871,
        0.4275727 , -0.2314186 ,  0.11620776, -0.78520846, -0.13842689,
        0.4584956 ,  0.21420497,  0.1071602 , -0.22134042, -0.9055725 ,
       -0.07672143, -0.7315552 , -0.5939882 ,  0.22811761,  0.4002627 ,
       -0.3080911 , -0.47884065, -0.1695623 ,  0.39910364,  0.3628818 ,
        0.41681492, -0.79096705, -0.01365972, -0.20807518, -0.41520938,
        0.26859868,  0.49305695, -0.12976363, -0.6548134 ,  0.28610456,
       -0.01905077, -0.11575596,  0.37693292,  0.0061153 , -0.23683777,
        0.82976   , -0.01315832,  0.857942  , -1.0047011 ,  0.66

#### Función para obtener vector promedio

In [140]:
#A través de esta función se saca el vector promedio caracteristico 
def getFV(documento):
    L = []
    words = documento.split()
    L = np.array([modelo_w2v.wv[w] for w in words if w in modelo_w2v.wv.key_to_index])
    if len(L):
        return np.mean(L,axis=0) #se saca el promedio
    else:
        return np.zeros(100)

In [141]:
getFV("odio gay")

array([-0.20250496,  0.793291  ,  0.01368177,  0.15764631,  0.21099919,
       -0.91225994,  0.66633964,  1.5281088 , -0.6237575 , -0.8987496 ,
       -0.11962215, -0.88339937,  0.02074496,  0.2111491 ,  0.11479285,
       -0.52059376,  0.36559218, -0.35429475, -0.34665126, -1.4926498 ,
        0.59418404,  0.32234916,  1.0212837 , -0.48982054,  0.00456946,
        0.15918067, -0.7093687 , -0.13978262, -0.7527528 ,  0.15001726,
        0.48150206, -0.12691979,  0.09807251, -1.1499437 , -0.2169351 ,
        0.70608014,  0.18238142,  0.14836538, -0.32120264, -1.0161685 ,
       -0.09691046, -0.8706307 , -0.6121837 ,  0.29985964,  0.46784088,
       -0.57806945, -0.5220212 , -0.1451973 ,  0.49517235,  0.44497454,
        0.4749779 , -0.7219528 , -0.05260438, -0.24259654, -0.45015216,
        0.33543378,  0.5793189 , -0.30161273, -0.8086684 ,  0.5383253 ,
       -0.09010372, -0.3082434 ,  0.510827  ,  0.10539587, -0.2589645 ,
        0.8931618 , -0.20231837,  0.9168234 , -1.0619857 ,  0.69

#### Aplicando la función getFV a todos los comentarios del dataset

In [142]:
#se saca los vectores caracteristicos a cada df tanto de train y test.
hate_train_FV = hate_train["Texto Limpio"].apply(lambda x: getFV(x))
no_hate_train_FV = no_hate_train["Texto Limpio"].apply(lambda x: getFV(x))
hate_test_FV = hate_test["Texto Limpio"].apply(lambda x: getFV(x))
no_hate_test_FV = no_hate_test["Texto Limpio"].apply(lambda x: getFV(x))

In [143]:
#Se estandariza las etiquetas a formato entero
hate_train["HATE"] = 1
hate_test["HATE"] = 1
no_hate_train["HATE"] = 0
no_hate_test["HATE"] = 0

In [144]:
no_hate_test_FV

12879    [-0.21483359, 0.6934173, 0.08286718, 0.1658685...
10954    [-0.15657318, 0.6691847, -0.07914188, 0.166235...
10304    [-0.23139583, 0.7821595, -0.00905841, 0.138436...
14192    [-0.111919336, 0.4961384, 0.026952114, 0.08916...
12150    [-0.17980418, 0.6535479, -0.018154966, 0.14522...
                               ...                        
13596    [-0.21397658, 0.6811969, 0.042979807, 0.188356...
12521    [-0.19500169, 0.690184, 0.03627778, 0.18370615...
10333    [-0.16445737, 0.91079265, -0.22124012, 0.12149...
12865    [-0.17456676, 0.44914293, 0.069868386, 0.12282...
14180    [-0.29311618, 0.7813847, 0.014124836, 0.238057...
Name: Texto Limpio, Length: 100, dtype: object

In [145]:
#Se junta la data de entrenamiento
X_train = np.concatenate((hate_train_FV.tolist(),no_hate_train_FV.tolist()),axis=0)
X_train.shape

(800, 100)

In [146]:
#Se junta las etiquetas de entrenamiento tanto hate y no hate
Y_train = np.concatenate((hate_train["HATE"].values,no_hate_train["HATE"].values),axis=0)
Y_train.shape

(800,)

In [147]:
#Se junta la data de test
X_test = np.concatenate((hate_test_FV.tolist(),no_hate_test_FV.tolist()),axis=0)
X_test.shape

(200, 100)

In [185]:
#Se junta las etiquetas del test
Y_test = np.concatenate((hate_test["HATE"].values,no_hate_test["HATE"].values),axis=0)
Y_test.shape

(200,)

## SVM

In [149]:
from sklearn import svm
from sklearn.metrics import confusion_matrix

In [175]:
#Se usa la funcion svm con parametros de ajuste, entrenamos
clasificador = svm.SVC(gamma=0.1,C=10)
clasificador.fit(X_train,Y_train)

SVC(C=10, gamma=0.1)

In [176]:
#predecimos nuestra variable de test
Y_pred = clasificador.predict(X_test)

In [177]:
Y_pred

array([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0], dtype=int64)

In [178]:
from sklearn.metrics import accuracy_score,classification_report

In [179]:
#Hallamos la precisión del modelo
accuracy_score(Y_test,Y_pred)

0.695

In [180]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.69      0.70      0.70       100
           1       0.70      0.69      0.69       100

    accuracy                           0.69       200
   macro avg       0.70      0.69      0.69       200
weighted avg       0.70      0.69      0.69       200



In [181]:
print(clasificador.predict(getFV("me gustaria que te mueras puto gay").reshape(1,-1)))

[1]


In [182]:
print(clasificador.predict(getFV("te quiero mucho").reshape(1,-1)))

[0]


In [186]:
#mostramos la matriz de confusión
confusion_matrix(Y_test,Y_pred)

array([[70, 30],
       [31, 69]], dtype=int64)